In [ ]:
!nvidia-smi

Mon May  9 20:02:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
%%capture
!wget http://images.cocodataset.org/zips/train2014.zip
!wget http://images.cocodataset.org/annotations/annotations_trainval2014.zip

In [2]:
%%capture
!git lfs install
!git clone https://huggingface.co/Cene655/Imagen-pytorch

In [ ]:
!pip install git+https://github.com/cene555/Imagen-pytorch.git

In [5]:
%%capture
!unzip /content/train2014.zip
!unzip /content/annotations_trainval2014.zip

In [6]:
from PIL import Image
from IPython.display import display
import torch as th
import pandas as pd
from imagen_pytorch.download import load_checkpoint
from imagen_pytorch.model_creation import (
    create_model_and_diffusion,
    model_and_diffusion_defaults,
)
from imagen_pytorch.resample import create_named_schedule_sampler

from imagen_pytorch import dist_util, logger
from imagen_pytorch.dataset import get_loader
from imagen_pytorch.train_utils import TrainLoop

has_cuda = th.cuda.is_available()
device = th.device('cpu' if not has_cuda else 'cuda')

In [ ]:
options = model_and_diffusion_defaults()
options['use_fp16'] = False
options['t5_name'] = 't5-large'
model, diffusion = create_model_and_diffusion(**options)
model.eval()
#if has_cuda:
#    model.convert_to_fp16()
model.to(device)
#model.load_state_dict(load_checkpoint('base', device), strict=False)
model.load_state_dict(th.load('/content/Imagen-pytorch/imagen-pytorch.pt'))

print('total base parameters', sum(x.numel() for x in model.parameters()))

In [8]:
def get_images_id(images_list):
    images_dict = {}
    for i in images_list:
        images_dict[i['id']] = i['file_name']
    return images_dict

In [9]:
import json

with open('/content/annotations/captions_train2014.json') as json_file:
    data = json.load(json_file)

In [10]:
images_dict = get_images_id(data['images'])

In [11]:
df = []
for annotation in data['annotations']:
    df.append([images_dict[annotation['image_id']], annotation['caption']])

In [12]:
df = pd.DataFrame(df)
df.columns = ['path', 'text']

In [14]:
data = get_loader(batch_size=4,
                  resolution=64,
                   image_dir='/content/train2014',
                   df=df,
                   tokenizer_name='t5-large', 
                   max_len=128,
                   zero_text_prob=0.1,
                   shuffle=True)

In [ ]:
next(data)

In [16]:
#a, b = next(data)

In [17]:
dist_util.setup_dist()
logger.configure()

logger.log("creating model and diffusion...")

Logging to /tmp/openai-2022-05-24-12-19-09-300494
creating model and diffusion...


In [18]:
schedule_sampler = create_named_schedule_sampler('uniform', diffusion)

In [ ]:
TrainLoop(
        model=model,
        diffusion=diffusion,
        data=data,
        batch_size=4,
        microbatch=-1,
        lr=1e-4,
        ema_rate="0.9999",
        log_interval=100,
        save_interval=10000,
        resume_checkpoint=False,
        use_fp16=False,
        fp16_scale_growth=1e-3,
        schedule_sampler=schedule_sampler,
        weight_decay=0.01,
        lr_anneal_steps=0,
        save_dir='/content/',
).run_loop()

In [ ]:
th.save(model.state_dict(), 'imagen_new.pt')